In [44]:
# library import
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import requests

import time
#from datetime import datetime
import re
import datetime

### 사용법 : 아래 input에 google id, email, pw, 검색어, maxpage, 시작날짜, 마지막 날짜를 형식에 맞게 입력하고 돌리면 됩니다

In [155]:
############################ 사용자 입력해야 하는 부분 #############################

your_id = 'cpark40'                 # 구글 아이디   (ex. 'abcde')     
your_email = 'cpark40@khu.ac.kr'    # 구글 이메일   (ex. 'abcde@khu.ac.kr')
your_pw = ''              # 구글 비밀번호 (ex. 'pw1234')

#keywords = "P+%26+G"                   # 검색어        (ex. 'nike')
keywords = 'nike'
maxpage = 5                        # 크롤링 하고 싶은 페이지 양  (ex. 20)

# 날짜 입력
start_date = datetime.datetime(2020,1,1) # 시작 날짜        (ex. 2020, 1,1) 
end_date = start_date
wanna_end_date = datetime.datetime(2020,10,1) # 마지막 날짜  (ex. 2020.11.1)

####################################################################################

In [156]:
# 파일명 제작용 카피본
s_date = start_date
e_date = wanna_end_date

month_31 = [1,3,5,7,8,10,12]
month_30 = [4,6,9,11]
month_28 = [2] # 2020.2는 윤년이다. # 연도가 4로 나누어 떨어질때는 29일까지 있음

count = 0

1. 실행 중 출력에 message가 떠도 놀라지 마세요. 에러 아님
2. 처음 실행할때 아래 while문이 에러가 나면 그대로 한번 더 실행해보세요. 그래도 안되면 연락 주십쇼

In [157]:
while ((end_date.year != wanna_end_date.year) or (end_date.month != wanna_end_date.month)):
    
    result = []
    
    # 크롬의 크롤링 방지를 피하기 위한 재로그인 자동화
    driver = webdriver.Chrome(executable_path='C:/Users/chaeyoung/Desktop/Django/myproject/chromedriver.exe')

    url = 'https:www.google.com'
    driver.get(url)

    #로그인
    search = driver.find_element_by_xpath('//*[@id="gb_70"]')
    search.send_keys(Keys.ENTER)

    search = driver.find_element_by_xpath('//*[@id="identifierId"]')
    search.send_keys(your_email)
    search.send_keys(Keys.ENTER)

    driver.implicitly_wait(30)

    search = driver.find_element_by_xpath('//*[@id="primaryId"]')
    search.send_keys(your_id)

    search = driver.find_element_by_xpath('//*[@id="password"]')
    search.send_keys(your_pw)

    search.send_keys(Keys.ENTER)

    driver.implicitly_wait(30)

    search = driver.find_element_by_class_name('VfPpkd-RLmnJb')
    search.click()
    
    # 날짜 전처리
    if (start_date.month in month_31):
        end_date = start_date + datetime.timedelta(days = 31)
    elif (start_date.month in month_30):
        end_date = start_date + datetime.timedelta(days = 30)
    elif ((start_date.month in month_28) and (start_date.year % 4 != 0)): # 윤년 아님
        end_date = start_date + datetime.timedelta(days = 28)
    elif ((start_date.month in month_28) and (start_date.year % 4 == 0)): # 윤년임
        end_date = start_date + datetime.timedelta(days = 29)
    else:
        print('날짜 error')
    
    start_month = str(start_date.month)
    start_day = str(start_date.day)
    start_year = str(start_date.year)

    end_month = str(end_date.month)
    end_day = str(end_date.day)
    end_year = str(end_date.year)
    
    print(start_year, start_month)
    print(end_year, end_month)
    print('-----------')
    
    # 크롤링 해서 제목을 title에 담는다. 
    title = []
    
    for j in range(0,maxpage * 10, 10):
        print(j)
        urls_new = "https://www.google.com/search?q="+keywords+"&hl=en&biw=768&bih=696&sxsrf=ALeKk00krMOwZt3eNOGfMw_8kUAL6q1O1Q%3A1606659890048&source=lnt&tbs=cdr%3A1%2Ccd_min%3A"+start_month+"%2F"+start_day+"%2F"+start_year+"%2Ccd_max%3A"+end_month+"%2F"+end_day+"%2F"+end_year+"&tbm=nws&ei=08TDX8L6HZXZhwObgJbgBw&start="+str(j)
        driver.get(urls_new)

        for i in range(1,11):
            try:
                x_path = '//*[@id="rso"]/div['+str(i)+']/g-card/div/div/div[2]/a/div/div[2]/div[2]'
                title.append(driver.find_element_by_xpath(x_path).text)
            except Exception as ex:
                print(ex)
        
        datee = start_year + "." + start_month
        
    dict_result = {'date' : datee,'title' : title}
    result.append(dict_result)

    start_date = end_date
    
    # 데이터 프레임으로 만들기
    x = []
    y = []

    for i in range(0,len(result)):
        for j in range(0,len(result[i]['title'])):
            x.append(result[i]['date'])
            y.append(result[i]['title'][j])
    
    # 처음 실행할때는 dataframe을 만들어주고, 그 다음부터는 기존 dataframe에 병합
    if (count == 0): # 처음 실행
        A = pd.DataFrame({'date':x, 'title' : y})
    elif (count > 0):
        B = pd.DataFrame({'date':x, 'title' : y})
        A = pd.concat([A,B], ignore_index=True)
    
    count = count + 1
    driver.close()

2020 1
2020 2
-----------
0
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[5]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.4240.198)

10
20
30
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[5]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.4240.198)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[9]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.4240.198)

40
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[6]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.4240.198)

2020 2
2020 3
-----------
0
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[1]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.424

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[7]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.4240.198)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[8]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.4240.198)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[9]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.4240.198)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[10]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.4240.198)

40
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[1]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.4240.198)

Message: no such element: Unable to locate element: {"me

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[7]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.4240.198)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[8]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.4240.198)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[9]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.4240.198)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[10]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.4240.198)

30
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[1]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.4240.198)

Message: no such element: Unable to locate element: {"me

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[10]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.4240.198)

20
Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[1]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.4240.198)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[2]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.4240.198)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[3]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.4240.198)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="rso"]/div[4]/g-card/div/div/div[2]/a/div/div[2]/div[2]"}
  (Session info: chrome=86.0.4240.198)

Message: no such element: Unable to locate element: {"me

KeyboardInterrupt: 

In [162]:
# 파일 저장
filename = "result/"+keywords +"_"+ str(s_date.year)+str(s_date.month)+"_"+str(e_date.year)+str(e_date.month)+".csv"

A.to_csv(filename, mode='w', index = False, encoding = 'utf-8')